In [5]:
import requests
import pandas as pd

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.WuKputPwZmA")

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content,'html.parser')

# Extracting a single element from data

seven_day = soup.find(id="seven-day-forecast") # tag that holds the weather info

forecast_items = seven_day.find_all(class_="tombstone-container") # forecast for the seven days

tonight = forecast_items[0] # forecast for the first day

period = tonight.find(class_="period_name") # tag for tonight
short_desc = tonight.find(class_="short-desc") # tag with short description
temp = tonight.find(class_="temp") # tag with temperature

img = tonight.find("img") # extract img tag
desc = img['title'] # extract the 'title' from img tag

# Extracting all elements from data

period_tags = seven_day.select(".tombstone-container .period-name") # uses css selectors to select period names
periods = [pt.get_text() for pt in period_tags] # iterate through period tags which hold which day it is
# same method can be used to access their descriptions
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

# now we use Pandas to analyze the data

weather = pd.DataFrame({ # we use the data frame class and pass in each list of items as a dictionary
    "Periods" : periods,
    "Short_descs" : short_descs,
    "Temps" : temps,
    "Descs" : descs
})




In [6]:
weather

,Descs,Periods,Short_descs,Temps
0,"Tonight: Partly cloudy, with a low around 51. ...",Tonight,Partly Cloudy,Low: 51 °F
1,"Monday: Mostly sunny, with a high near 61. Wes...",Monday,Mostly Sunny,High: 61 °F
2,"Monday Night: Mostly clear, with a low around ...",MondayNight,Mostly Clear,Low: 50 °F
3,"Tuesday: Sunny, with a high near 65. West wind...",Tuesday,Sunny,High: 65 °F
4,"Tuesday Night: Mostly clear, with a low around...",TuesdayNight,Mostly Clear,Low: 51 °F
5,"Wednesday: Sunny, with a high near 66.",Wednesday,Sunny,High: 66 °F
6,"Wednesday Night: Mostly clear, with a low arou...",WednesdayNight,Mostly Clear,Low: 52 °F
7,"Thursday: Mostly sunny, with a high near 65.",Thursday,Mostly Sunny,High: 65 °F
8,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudy,Low: 52 °F


In [7]:
temp_nums = weather["Temps"].str.extract("(?P<temp_num>\d+)", expand=False)

In [8]:
weather["temp_num"] = temp_nums.astype('int')

In [9]:
temp_nums

0    51
1    61
2    50
3    65
4    51
5    66
6    52
7    65
8    52
Name: temp_num, dtype: object

In [11]:
weather["temp_num"].mean()

57.0

In [14]:
is_night = weather["Temps"].str.contains("Low")

In [15]:
weather["is_night"] = is_night

In [16]:
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: Temps, dtype: bool

In [ ]:
weather[is_night]